### Importar librerías

In [9]:
import pandas as pd 
import sqlalchemy
import missingno as msn 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import numpy as np

### Carga de datos

In [10]:
df = pd.read_csv('../Data/df.csv')
#DROP COLUMNS
df = df.drop(columns=['adjustSquareMeterPrice', 'automaticACMPrice', 'created_at', 'terraceValue',
                       'utilityRoomValue', 'preofferAcmHigherValue','preofferAcmLowerValue','buildingFloors','estateFloors',
                       'rentValue','garageType','terraceArea','zone','garageValue', 'type'])

In [11]:
print('la dimension del dataset es :',df.shape)

la dimension del dataset es : (18911, 12)


### Definir funcion de preprocesamiento

In [12]:
def preprocesamiento(df):
    #Drop rows with null values in Y variable
    df = df[df.finalACMPrice.notna()]
    #Calculate Antiguedad of a property in years
    df['Antiguedad'] = date.today().year - df['year']
    df['Antiguedad'] = df['Antiguedad'].astype(int)
    df = df.drop(columns=['id', 'purchaseCaseId', 'year'])
    #delete rows with null values in stratum variables
    df = df[df.stratum.notna()]
    #Turn floor to integer
    df = df[df.floor.notna()]
    df['floor'] = df['floor'].astype(int)
    #df = df[df.stratum.notna()]
    # Convert hasElevator to categorical
    df['hasElevator'] = df['hasElevator'].astype('category')
    return df

df = preprocesamiento(df)

C:\Users\User\AppData\Local\Temp\ipykernel_1680\1233199684.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Antiguedad'] = date.today().year - df['year']
C:\Users\User\AppData\Local\Temp\ipykernel_1680\1233199684.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Antiguedad'] = df['Antiguedad'].astype(int)


### Ejemplo del datset preprocesado

In [13]:
df.head(30)

,finalACMPrice,area,bathrooms,hasElevator,floor,stratum,garages,latitude,longitude,Antiguedad
12,1.350139e+08,53.0,1,1.0,11,3.0,0,4.651472,-74.170378,5
15,1.321759e+08,47.0,1,1.0,22,3.0,0,6.285643,-75.610017,4
19,1.301991e+08,43.0,1,0.0,3,2.0,0,4.644194,-74.157111,16
27,1.387801e+08,52.0,2,0.0,4,2.0,1,4.631931,-74.205299,9
38,1.121298e+08,42.0,1,1.0,19,3.0,0,6.182831,-75.648336,11
44,4.094016e+08,68.0,2,1.0,6,4.0,1,4.738678,-74.056186,5
45,1.287481e+08,41.0,1,1.0,16,3.0,1,6.323325,-75.547640,2
49,5.082177e+08,113.0,3,1.0,7,5.0,2,6.176877,-75.576370,14
64,1.873527e+08,58.0,2,1.0,14,3.0,1,6.324183,-75.569169,4
68,3.782191e+08,147.0,3,1.0,2,5.0,1,6.220482,-75.602549,14
